# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [1]:
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Analyzer Templates

Below is a collection of analyzer templates designed to extract fields from various input file types.

These templates are highly customizable, allowing you to modify them to suit your specific needs. For additional verified templates from Microsoft, please visit [here](../analyzer_templates/README.md).

In [2]:
extraction_templates = {
    "PreTranscribedCallAnalyticsOnText":  ('../CU-Demo-Assets/analyzers/PostCallAnalyticsOnText.json', '../CU-Demo-Assets/en-us-conversation-transcripts/sample0 - Insurance - Customer calls about purchasing life insurance.wav.results.txt'),
    "ConversationAnalyticsOnText":  ('../CU-Demo-Assets/analyzers/PostCallAnalyticsOnText.json', '../CU-Demo-Assets/en-us-conversation-transcripts/customer-sample4.txt'),
    "MultiLanguageDemoOnText":  ('../CU-Demo-Assets/analyzers/MultiLanguageDemoOnText.json', '../CU-Demo-Assets/pt-BR-conversation-transcripts/sample6 - Financial - Customer calls about transferring funds - Brazilian.wav.results.txt'),
    "StoryAnalysisTXT": ('../CU-Demo-Assets/analyzers/ShortStoryAnalyzer.json', '../CU-Demo-Assets/en-us-text-examples/extended_short_story.txt'),
    "StoryAnalysisHTML": ('../CU-Demo-Assets/analyzers/ShortStoryAnalyzer.json', '../CU-Demo-Assets/en-us-text-examples/extended_short_story.html'),
    "StoryAnalysisWObject": ('../CU-Demo-Assets/analyzers/ShortStoryAnalyzerWObject.json', '../CU-Demo-Assets/en-us-text-examples/extended_short_story.html'),
    "QnA-Extraction": ('../CU-Demo-Assets/analyzers/QnA-Extraction.json', '../CU-Demo-Assets/en-us-text-examples/EmailBody1.txt')
}

Specify the analyzer template you want to use and provide a name for the analyzer to be created based on the template.

In [3]:
import uuid

ANALYZER_TEMPLATE = "QnA-Extraction"
ANALYZER_ID = "cu-demo-" + str(uuid.uuid4())

(analyzer_template_path, analyzer_sample_file_path) = extraction_templates[ANALYZER_TEMPLATE]

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [4]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/field_extraction", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.36)'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 400
Response headers:
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'IMDS/150.870.65.1544'
    'x-ms-request-id': 'e2f1f7ca-2845-41c4-8f14-c7ff69f6eb8c'
    'Date': 'Mon, 21 Apr 2025 19:53:57 GMT'
    'Content-Length': '88'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Re

## List all analyzers created in your resource

After the analyzer is successfully created, we can use it to analyze our input files.

In [5]:
response = client.get_all_analyzers()
print(f"Number of analyzers in your resource: {len(response['value'])}")
print(f"First 3 analyzer details: {json.dumps(response['value'], indent=2)}")

Number of analyzers in your resource: 8
First 3 analyzer details: [
  {
    "analyzerId": "prebuilt-read",
    "description": "Extract content elements such as words, barcodes, and formulas from documents.",
    "config": {
      "returnDetails": true,
      "enableOcr": true,
      "enableLayout": false,
      "enableBarcode": false,
      "enableFormula": false
    },
    "warnings": [],
    "status": "undefined",
    "scenario": "document"
  },
  {
    "analyzerId": "prebuilt-layout",
    "description": "Extract various content and layout elements such as words, paragraphs, and tables from documents.",
    "config": {
      "returnDetails": true,
      "enableOcr": true,
      "enableLayout": true,
      "enableBarcode": false,
      "enableFormula": false
    },
    "warnings": [],
    "status": "undefined",
    "scenario": "document"
  },
  {
    "analyzerId": "TempTest",
    "description": "",
    "tags": {
      "projectId": "eaf9a896-8d9c-46ae-8cfd-e3f601ae388f",
      "templat

## Create Analyzer from the Template

In [9]:
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzer cu-demo-d7caa897-3a0f-4192-b757-fe549f4bace9 create request accepted.
INFO:python.content_understanding_client:Request 8532467c-06a2-46b1-ac22-d3ea12f4368d in progress ...
INFO:python.content_understanding_client:Request result is ready after 2.71 seconds.


{
  "id": "8532467c-06a2-46b1-ac22-d3ea12f4368d",
  "status": "Succeeded",
  "result": {
    "analyzerId": "cu-demo-d7caa897-3a0f-4192-b757-fe549f4bace9",
    "description": "Extraction of questions and answers from emails.",
    "tags": {
      "projectId": "16f1d030-0fac-4ce0-a92c-88bcd6eee3d8",
      "templateId": "text-2024-12-01"
    },
    "createdAt": "2025-04-21T19:55:04Z",
    "lastModifiedAt": "2025-04-21T19:55:05Z",
    "config": {
      "returnDetails": false,
      "disableContentFiltering": false
    },
    "fieldSchema": {
      "fields": {
        "QnAPair": {
          "type": "array",
          "method": "generate",
          "description": "A list of questions and anwers asked in the email.",
          "items": {
            "type": "object",
            "properties": {
              "Question": {
                "type": "string",
                "description": "Question asked in the email."
              },
              "Answer": {
                "type": "string",

# Extract Conversation (WEBVTT) from either Batch Transcription JSON or Content Understanding JSON

## Extract Fields Using the Analyzer

After the analyzer is successfully created, we can use it to analyze our input files.

In [10]:
response = client.begin_analyze(ANALYZER_ID, file_location=analyzer_sample_file_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzing file ../CU-Demo-Assets/en-us-text-examples/EmailBody1.txt with analyzer: cu-demo-d7caa897-3a0f-4192-b757-fe549f4bace9
INFO:python.content_understanding_client:Request f8f3473b-2d91-4f2b-9499-35bb197ed2c1 in progress ...
INFO:python.content_understanding_client:Request result is ready after 2.91 seconds.


{
  "id": "f8f3473b-2d91-4f2b-9499-35bb197ed2c1",
  "status": "Succeeded",
  "result": {
    "analyzerId": "cu-demo-d7caa897-3a0f-4192-b757-fe549f4bace9",
    "apiVersion": "2024-12-01-preview",
    "createdAt": "2025-04-21T19:55:13Z",
    "warnings": [],
    "contents": [
      {
        "markdown": "```\nHi Josh,\r\n\r\n \r\n\r\nThanks for the background.\r\n\r\nThe `speechStartDetected` event should come first, but usually only by a few ms if not at the same time with the first hypothesis (onRecognizing event).\r\n\r\nYou\u2019ll also get one for each turn, or reconnection.\r\n\r\n \r\n\r\nHope that helps,\r\n\r\n              Heiko\r\n\r\n \r\n\r\nFrom: Josh Schraivogel <Josh.Schraivogel@genesys.com> \r\nSent: Friday, April 11, 2025 10:48 AM\r\nTo: Heiko Rahmel <Heiko.Rahmel@microsoft.com>\r\nSubject: [EXTERNAL] Re: Microsoft ASR event speechStartDetected\r\n\r\n \r\n\r\nThank you so much Heiko!\r\n\r\n \r\n\r\nAfter some of the performance hiccups we have been having with STT Engi

## Clean Up
Optionally, delete the sample analyzer from your resource. In typical usage scenarios, you would analyze multiple files using the same analyzer.

In [11]:
client.delete_analyzer(ANALYZER_ID)

INFO:python.content_understanding_client:Analyzer cu-demo-d7caa897-3a0f-4192-b757-fe549f4bace9 deleted.


<Response [204]>